In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

In [2]:
fes = H1(mesh, order=4, dirichlet="left|bottom")
u,v = fes.TnT()
gfu = GridFunction(fes)

wind = CF( (1, 0.2) )
gfu.Set (exp(-100*( (x-0.2)**2 + (y-0.5)**2) ))

In [3]:
n = specialcf.normal(2)
Atrans = BilinearForm(-u*wind*grad(v)*dx + wind*n*u*v*ds).Assemble()
f = LinearForm(fes)
mass = BilinearForm(u*v*dx).Assemble()
massinv = mass.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")

In [4]:
scene = Draw(gfu, min=0, max=1)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [5]:
dt = 0.0002
tend = 0.8
t = 0
while t < tend:
    gfu.vec.data += dt * massinv * (f.vec-Atrans.mat*gfu.vec)
    t += dt
    scene.Redraw()
    

## Diffferent wind

In [6]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
wind = CF( (1, 0.1 * sin(30 * x)) )
gfu.Set (exp(-100*( (x-0.2)**2 + (y-0.5)**2) ))

In [7]:
n = specialcf.normal(2)
Atrans = BilinearForm(-u*wind*grad(v)*dx + wind*n*u*v*ds).Assemble()
f = LinearForm(fes)
mass = BilinearForm(u*v*dx).Assemble()
massinv = mass.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")

In [8]:
scene = Draw(gfu, min=0, max=1)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [9]:
dt = 0.0002
tend = 0.8
t = 0
while t < tend:
    gfu.vec.data += dt * massinv * (f.vec-Atrans.mat*gfu.vec)
    t += dt
    scene.Redraw()

### result:
    now movement is not horizontal desplacement it has a really tiny osilation because of the sin

### Make it not sin

In [49]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
wind = CF( (-(y-0.5), x-0.5) )
gfu.Set (exp(-100*( (x-0.2)**2 + (y-0.5)**2) ))

In [50]:
n = specialcf.normal(2)
Atrans = BilinearForm(-u*wind*grad(v)*dx + wind*n*u*v*ds).Assemble()
f = LinearForm(fes)
mass = BilinearForm(u*v*dx).Assemble()
massinv = mass.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")

In [51]:
scene = Draw(gfu, min=0, max=1)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [53]:
dt = 0.0002
tend = 0.8
t = 0
while t < tend:
    gfu.vec.data += dt * massinv * (f.vec-Atrans.mat*gfu.vec)
    t += dt
    scene.Redraw()

## improve timestepping to order Runge Kutta

In [56]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
wind = CF( (1, 1.0 * sin(30 * x)) )
gfu.Set (exp(-100*( (x-0.2)**2 + (y-0.5)**2) ))
n = specialcf.normal(2)
Atrans = BilinearForm(-u*wind*grad(v)*dx + wind*n*u*v*ds).Assemble()
f = LinearForm(fes)
mass = BilinearForm(u*v*dx).Assemble()
massinv = mass.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")
scene = Draw(gfu, min=0, max=1)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [57]:
dt = 0.0002
tend = 0.8
t = 0
k1 = gfu.vec.CreateVector()
k2 = gfu.vec.CreateVector()
u_pred = gfu.vec.CreateVector()
while t < tend:
    k1.data = massinv * (f.vec - Atrans.mat * gfu.vec)
    u_pred.data = gfu.vec + dt * k1
    k2.data = massinv * (f.vec - Atrans.mat * u_pred)
    gfu.vec.data += (dt/2) * (k1 + k2)
    t += dt
    scene.Redraw()

## Add a diffusion term to the transport equation:

In [15]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
fes = H1(mesh, order=4, dirichlet="left|bottom")
u,v = fes.TnT()
gfu = GridFunction(fes)

wind = CF( (1, 0.2) )
gfu.Set (exp(-100*( (x-0.2)**2 + (y-0.5)**2) ))

In [16]:
eps = 0.001
n = specialcf.normal(2)
Atrans = BilinearForm(-u*wind*grad(v)*dx + wind*n*u*v*ds).Assemble()
f = LinearForm(fes)
mass = BilinearForm(u*v*dx).Assemble()
massinv = mass.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")

# 1. Convection Term: 
conv = BilinearForm(fes)
conv += -wind * u * grad(v) * dx
# outflow boundary
conv += wind * n * u * v * ds(definedon=mesh.Boundaries("right|top"))
# 2. Diffusion Term
laplace = BilinearForm(fes)
laplace += eps * grad(u) * grad(v) * dx
mass = BilinearForm(u*v*dx)
f_source = LinearForm(fes)
f_source.Assemble()

conv.Assemble()
laplace.Assemble()
mass.Assemble()
massinv = mass.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")

# Combine convection and diffusion matrices
A_eff = laplace.mat + conv.mat 
scene = Draw(gfu, min=0, max=1)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [17]:
dt = 0.0002
tend = 0.8
t = 0
last_update_t = t
while t < tend:
    rhs_vec = f_source.vec - A_eff * gfu.vec
    gfu.vec.data += dt * massinv * rhs_vec
    t_old = t
    t += dt
    scene.Redraw()

### result:

looks kinda as expected not only it moves horizontaly due to convection but we add a diffusion term so it almost doesn't reach to the corner without disapearing

## add a source term 

In [22]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
fes = H1(mesh, order=4, dirichlet="left|bottom")
u,v = fes.TnT()
gfu = GridFunction(fes)

wind = CF( (1, 0.2) )
gfu.Set (exp(-100*( (x-0.2)**2 + (y-0.5)**2) ))

In [23]:
eps = 0.001
n = specialcf.normal(2)
Atrans = BilinearForm(-u*wind*grad(v)*dx + wind*n*u*v*ds).Assemble()
f = LinearForm(fes)
mass = BilinearForm(u*v*dx).Assemble()
massinv = mass.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")

# 1. Convection Term: 
conv = BilinearForm(fes)
conv += -wind * u * grad(v) * dx
# outflow boundary
conv += wind * n * u * v * ds(definedon=mesh.Boundaries("right|top"))
# 2. Diffusion Term
laplace = BilinearForm(fes)
laplace += eps * grad(u) * grad(v) * dx
mass = BilinearForm(u*v*dx)
f_source = LinearForm(fes) 
source_cf = sin(pi*x) * sin(pi*y)
f_source += source_cf * v * dx
f_source.Assemble()

conv.Assemble()
laplace.Assemble()
mass.Assemble()
massinv = mass.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")

# Combine convection and diffusion matrices
A_eff = laplace.mat + conv.mat 
scene = Draw(gfu, min=0, max=1)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [24]:
dt = 0.0002
tend = 0.8
t = 0
last_update_t = t
while t < tend:
    rhs_vec = f_source.vec - A_eff * gfu.vec
    gfu.vec.data += dt * massinv * rhs_vec
    t_old = t
    t += dt
    scene.Redraw()